In [ ]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf


tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=20000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)



if __name__ == "__main__":
  tf.app.run()


Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 2.3100533, step = 2
INFO:tensorflow:probabilities = [[0.09719655 0.10516629 0.08253608 0.10296774 0.09293492 0.09703191
  0.10050346 0.10939199 0.09791633 0.11435465]
 [0.0897489  0.11360925 0.08185133 0.11138219 0.11034381 0.09127079
  0.09787449 0.08551478

INFO:tensorflow:probabilities = [[0.09226216 0.1141763  0.08097399 0.10841182 0.10187573 0.08726814
  0.09780225 0.10723067 0.09184295 0.11815599]
 [0.08166129 0.10440721 0.08196403 0.10184147 0.109363   0.09899648
  0.10569737 0.0911731  0.1011814  0.12371463]
 [0.09697298 0.113399   0.09164407 0.09905379 0.10593531 0.09113691
  0.09292994 0.08903897 0.09523791 0.12465117]
 [0.0974678  0.12191225 0.08407174 0.10682968 0.1061499  0.07795919
  0.10950291 0.09239627 0.0939085  0.10980174]
 [0.09837032 0.09926294 0.10127968 0.10859004 0.10303251 0.09247861
  0.10713506 0.09085346 0.09221123 0.10678611]
 [0.10045318 0.11752419 0.08704588 0.09319229 0.10794237 0.08369331
  0.10686314 0.10915172 0.09782603 0.09630784]
 [0.09187321 0.11039281 0.10064603 0.10367783 0.09270921 0.08863726
  0.1143709  0.10290251 0.08880341 0.10598679]
 [0.09936034 0.11605513 0.07471675 0.11147401 0.10385824 0.08812857
  0.10753445 0.09216421 0.09807295 0.10863534]
 [0.09285126 0.11398603 0.09978656 0.09516943 0.

INFO:tensorflow:global_step/sec: 2.2841
INFO:tensorflow:loss = 2.2979136, step = 102 (43.782 sec)
INFO:tensorflow:probabilities = [[0.11571237 0.10134918 0.09869896 0.09814168 0.09094462 0.09426256
  0.10657388 0.09534279 0.08628925 0.11268471]
 [0.09928493 0.11541784 0.08594357 0.10759591 0.10153243 0.08175077
  0.09932836 0.10984261 0.09113686 0.10816669]
 [0.09931904 0.09120693 0.09387017 0.11260236 0.09683578 0.08841823
  0.10206792 0.09457918 0.10286842 0.11823203]
 [0.09721345 0.1067718  0.09560347 0.10744618 0.1007413  0.08988766
  0.10990635 0.08622539 0.10440426 0.10180008]
 [0.10442372 0.1117217  0.08611293 0.09746458 0.11148858 0.09039088
  0.09411366 0.10179118 0.09588209 0.10661066]
 [0.0919834  0.1129525  0.08652668 0.11839882 0.10001945 0.09623054
  0.10570534 0.09130192 0.08790022 0.10898117]
 [0.09090663 0.10761011 0.09430135 0.0964281  0.09717499 0.10865167
  0.11142907 0.0902396  0.10190635 0.10135216]
 [0.10657093 0.10317512 0.09126719 0.11367583 0.09524927 0.089540

INFO:tensorflow:probabilities = [[0.09453046 0.10653812 0.09266372 0.10008582 0.09623726 0.09804595
  0.12094485 0.07378192 0.10858092 0.10859095]
 [0.08717868 0.10310042 0.08632714 0.11520863 0.09692858 0.09186414
  0.1122839  0.10224329 0.0919923  0.11287296]
 [0.10477054 0.09997753 0.10156967 0.08831621 0.10364925 0.09782362
  0.10661476 0.09060714 0.10445894 0.10221234]
 [0.09579404 0.1124689  0.09319282 0.11590989 0.09390422 0.08768944
  0.10038828 0.09054102 0.097155   0.1129564 ]
 [0.08998112 0.09584787 0.09375995 0.11749061 0.10623634 0.08802483
  0.11058182 0.08005367 0.1103922  0.10763166]
 [0.08924553 0.11929184 0.08487396 0.10641993 0.09316395 0.07422802
  0.09281617 0.11337587 0.11596204 0.11062262]
 [0.09358697 0.09571019 0.09226866 0.10738295 0.10253886 0.09583957
  0.11329713 0.0931595  0.10766359 0.09855261]
 [0.09473669 0.09751762 0.0866484  0.11615741 0.10180631 0.0941458
  0.08951019 0.10318246 0.11628381 0.1000114 ]
 [0.09174724 0.09979598 0.09139779 0.09777243 0.0

INFO:tensorflow:global_step/sec: 2.42401
INFO:tensorflow:loss = 2.2675855, step = 202 (41.259 sec)
INFO:tensorflow:probabilities = [[0.0886044  0.09830636 0.09169438 0.1080232  0.10164271 0.0904296
  0.09957547 0.10240966 0.09995445 0.11935978]
 [0.09831785 0.11820859 0.07803582 0.10621325 0.10086468 0.10116988
  0.10061984 0.09620352 0.09779339 0.10257314]
 [0.11425558 0.09337322 0.09037631 0.10855039 0.09810646 0.09532375
  0.10571242 0.0999158  0.09456739 0.09981868]
 [0.09500695 0.09858385 0.09562469 0.0992339  0.09701859 0.09529953
  0.11238588 0.10357071 0.10436048 0.09891538]
 [0.0999961  0.10043424 0.0910438  0.11830306 0.09529091 0.10391427
  0.10810763 0.07951359 0.10050105 0.10289536]
 [0.09511042 0.1041353  0.09620842 0.11056427 0.08801749 0.08513848
  0.09775759 0.10945714 0.09719067 0.11642029]
 [0.09906216 0.09987669 0.09096674 0.11292543 0.08239502 0.09570496
  0.09990912 0.11131235 0.09936294 0.1084846 ]
 [0.09958259 0.0982017  0.08489064 0.10579099 0.09408817 0.089729

...

INFO:tensorflow:probabilities = [[0.02295454 0.00904988 0.03970345 0.03039857 0.31550267 0.03406121
  0.10482761 0.1440623  0.07264705 0.2267927 ]
 [0.0391429  0.32578525 0.08391552 0.08124924 0.03716122 0.04992239
  0.09929881 0.11222953 0.0919417  0.07935344]
 [0.13436387 0.05752159 0.24056883 0.08213925 0.04917615 0.08400021
  0.1859789  0.01421341 0.13345645 0.01858125]
 [0.02818897 0.2470676  0.06412785 0.07770412 0.08636165 0.08447713
  0.10428317 0.0985437  0.12058394 0.08866189]
 [0.05902722 0.06884344 0.04623073 0.05267602 0.14120574 0.12077191
  0.1545196  0.04971147 0.14115976 0.16585408]
 [0.11749189 0.03007718 0.16227742 0.05334784 0.12884568 0.10290585
  0.23182929 0.01977847 0.09649219 0.05695416]
 [0.06628948 0.01530748 0.07586897 0.0742054  0.19650081 0.04035463
  0.08733239 0.14244834 0.10772641 0.19396612]
 [0.12642182 0.07136285 0.13866636 0.09050025 0.07138353 0.07799464
  0.1968883  0.0540515  0.13483801 0.03789274]
 [0.05210948 0.1917367  0.11122308 0.08821338 0.

INFO:tensorflow:global_step/sec: 2.15834
INFO:tensorflow:loss = 1.3047202, step = 1302 (46.332 sec)
INFO:tensorflow:probabilities = [[0.08041792 0.00930623 0.2558679  0.06431475 0.1436349  0.05288646
  0.20555162 0.03963793 0.08572897 0.06265336]
 [0.09563719 0.05591778 0.05041988 0.26077256 0.04777476 0.18605731
  0.06189119 0.04997609 0.1473263  0.04422696]
 [0.06481037 0.01527819 0.12493861 0.07202323 0.0799223  0.082931
  0.28727376 0.03285023 0.1541884  0.08578385]
 [0.0734429  0.04274163 0.12729281 0.05810023 0.16007002 0.04962186
  0.30655786 0.04411992 0.07503366 0.06301907]
 [0.01954047 0.04469736 0.30668443 0.08988528 0.05965231 0.05466209
  0.11630671 0.09822521 0.12122072 0.08912538]
 [0.05566528 0.02114876 0.06732947 0.09171617 0.07207348 0.04005077
  0.02703025 0.41118127 0.05362587 0.16017866]
 [0.01599333 0.06149062 0.02315653 0.05599124 0.36831525 0.06717236
  0.10404035 0.05399301 0.11099984 0.13884749]
 [0.03976422 0.1550479  0.07304195 0.11593431 0.08088342 0.177670

INFO:tensorflow:probabilities = [[0.0213151  0.41088554 0.05347104 0.06261744 0.03995978 0.08674911
  0.11314392 0.03261148 0.11327822 0.06596831]
 [0.0557138  0.03530388 0.1007792  0.2659091  0.0346604  0.11289162
  0.14855136 0.06107567 0.10355837 0.08155663]
 [0.02543805 0.04554814 0.04163512 0.10799605 0.03333124 0.06238045
  0.32354343 0.09058199 0.15618084 0.11336477]
 [0.0133258  0.5106465  0.03535564 0.04551244 0.03209376 0.04565425
  0.09304623 0.0530209  0.12876269 0.04258173]
 [0.01466192 0.44005647 0.0468346  0.08876488 0.0246973  0.07108202
  0.07770985 0.06904677 0.06212604 0.10502011]
 [0.07260137 0.02892141 0.600254   0.06604648 0.06615733 0.01934226
  0.04821438 0.00915515 0.05729954 0.03200803]
 [0.0363235  0.08801451 0.34368694 0.11938617 0.00644946 0.0712152
  0.0968596  0.05851589 0.14617865 0.03337013]
 [0.04195705 0.06670068 0.07359859 0.13176174 0.08753629 0.14659873
  0.09012906 0.05812087 0.12423828 0.17935875]
 [0.0589737  0.08833522 0.4121833  0.07864161 0.0

INFO:tensorflow:global_step/sec: 2.37655
INFO:tensorflow:loss = 1.2393471, step = 1402 (42.077 sec)
INFO:tensorflow:probabilities = [[0.02952164 0.1116124  0.12165248 0.04457242 0.06506795 0.10722064
  0.36555097 0.01094753 0.09132276 0.0525313 ]
 [0.04025828 0.0195063  0.1255339  0.4570399  0.04124488 0.16739875
  0.05252898 0.01629298 0.04458611 0.03561001]
 [0.02757437 0.13147625 0.09178354 0.08358587 0.06132197 0.08803158
  0.07538519 0.16314608 0.098844   0.17885119]
 [0.01657152 0.63009197 0.03658728 0.04130858 0.03243933 0.02568308
  0.02742456 0.0628479  0.08484989 0.04219585]
 [0.06712717 0.07754362 0.05699816 0.07479237 0.13785405 0.12802589
  0.06449524 0.0908154  0.14581735 0.1565308 ]
 [0.04810071 0.0162432  0.02937092 0.01259191 0.0626237  0.15591362
  0.0401039  0.21027623 0.24176887 0.18300685]
 [0.0173786  0.31084388 0.08134925 0.20208837 0.02362405 0.08154354
  0.05109701 0.0371158  0.15668568 0.03827382]
 [0.04388976 0.07102763 0.24752612 0.06043901 0.20939279 0.0912

INFO:tensorflow:Saving checkpoints for 1420 into /tmp/mnist_convnet_model/model.ckpt.


In [1]:
print("a")
a='www'
print(a)

a
www


In [ ]:
print(a)

In [ ]:
print("this var is %s".format(a))